In [1]:
import pandas as pd
import re
import pickle
from urllib.request import Request, urlopen
import bs4 as bs
import lxml
import json
import random
import pickle
import random
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import ftfy

#### Filter csv

In [2]:
df = pd.read_csv("../Data/filtered_df_muckrack_with_twitter8april2021.csv")

In [3]:
for i in range(len(df)):
    journos = json.loads(df.iloc[i].journalist)
    if len(journos) < 3:
        df.loc[i, "journalist"] = 'delete'

In [4]:
df = df[df.journalist != 'delete'].reset_index()

#### Get Alexa ranking

In [5]:
df.cleaned_outlet_url

0                  cbc.ca
1                  cbc.ca
2                  cbc.ca
3                  cbc.ca
4                  cbc.ca
              ...        
1100    opendemocracy.net
1101       nbcbayarea.com
1102      foxbusiness.com
1103              upi.com
1104         collider.com
Name: cleaned_outlet_url, Length: 1105, dtype: object

In [6]:
def get_alexa_ranking(url):
    try:
        xml = urlopen('http://data.alexa.com/data?cli=10&dat=s&url={}'.format(url)).read() 
        text = int(re.search(r'<POPULARITY[^>]*TEXT="(\d+)"', str(xml)).groups()[0])
    except:
        text = "Cannot find"
    return text

In [7]:
df['alexa_ranking'] = df.apply(lambda x: get_alexa_ranking(x['cleaned_outlet_url']), axis=1)

In [8]:
df = df[df.alexa_ranking != "Cannot find"].reset_index()

#### Get mean and median values

In [9]:
df = df[df.t_followers.notnull()]

In [10]:
def clean_followers(num):
    return int(num.replace(',', ''))

In [11]:
df['clean_t_followers'] = df.apply(lambda x: clean_followers(x['t_followers']), axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017 entries, 0 to 1096
Data columns (total 45 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   level_0                      1017 non-null   int64 
 1   index                        1017 non-null   int64 
 2   Accepts contributed content  53 non-null     object
 3   Broadcast Affiliation        63 non-null     object
 4   Country                      1013 non-null   object
 5   Days Published               289 non-null    object
 6   Frequency                    479 non-null    object
 7   Instagram                    710 non-null    object
 8   Language                     1016 non-null   object
 9   Media Market                 219 non-null    object
 10  Pinterest                    326 non-null    object
 11  Radio Format                 90 non-null     object
 12  Radio Media Market           9 non-null      object
 13  Scope                        1017

In [13]:
df.clean_t_followers.mean()

1328313.3038348083

In [14]:
df.clean_t_followers.median()

198676.0

In [15]:
df.alexa_ranking.median()

4093.0

In [16]:
df.alexa_ranking.mean()

13446.89872173058

#### Cutoff by median

In [17]:
select_df = df[df.clean_t_followers >= df.clean_t_followers.median()]
select_df = select_df[select_df.alexa_ranking <= df.alexa_ranking.median()]

In [18]:
select_df = select_df.drop(columns=['level_0', 'index'])

In [19]:
select_df = select_df.reset_index()

In [20]:
select_df.muckrack_profile_url

0      https://muckrack.com/media-outlet/cbc-saskatch...
1        https://muckrack.com/media-outlet/cbc-toronto-8
2          https://muckrack.com/media-outlet/cbc_toronto
3           https://muckrack.com/media-outlet/cbc_ottawa
4            https://muckrack.com/media-outlet/cbcottawa
                             ...                        
278                https://muckrack.com/media-outlet/nfl
279            https://muckrack.com/media-outlet/infobae
280           https://muckrack.com/media-outlet/euronews
281                https://muckrack.com/media-outlet/nhk
282        https://muckrack.com/media-outlet/foxbusiness
Name: muckrack_profile_url, Length: 283, dtype: object

#### Pick up random rows + random authors

In [21]:
select_df

,index,Accepts contributed content,Broadcast Affiliation,Country,Days Published,Frequency,Instagram,Language,Media Market,Pinterest,...,t_profile_website,t_tweet,t_twitter_username,t_verified,twitter,url,youtube,multiple_url,alexa_ranking,clean_t_followers
0,0,NaN,NaN,Canada,NaN,NaN,https://instagram.com/cbc,English,NaN,NaN,...,http://www.cbcnews.ca,Protected Account,@CBCNews,Verified account,https://twitter.com/CBCNews,http://cbc.ca/news/canada/saskatchewan,https://youtube.com/user/CBCtv,y,1316,3078927
1,2,NaN,NaN,Canada,NaN,NaN,https://instagram.com/cbc,English,NaN,NaN,...,http://www.cbcnews.ca,Protected Account,@CBCNews,Verified account,https://twitter.com/CBCNews,http://cbc.ca/news/canada/toronto,https://youtube.com/user/CBCtv,y,1316,3078927
2,3,NaN,NaN,Canada,NaN,NaN,https://instagram.com/cbc,English,NaN,NaN,...,http://www.cbc.ca/toronto,Protected Account,@CBCToronto,Verified account,https://twitter.com/CBCToronto,http://cbc.ca/news/canada/toronto,https://youtube.com/user/CBCtv,y,1316,606229
3,6,NaN,NaN,Canada,NaN,NaN,https://www.instagram.com/cbcottawa/?hl=en,English,NaN,NaN,...,http://cbc.ca/ottawa,Protected Account,@CBCOttawa,Verified account,https://twitter.com/CBCOttawa,http://cbc.ca/news/canada/ottawa,https://youtube.com/user/CBCtv,y,1316,241485
4,10,NaN,NaN,Canada,NaN,Weekly,https://www.instagram.com/cbcottawa/,English,NaN,NaN,...,http://cbc.ca/ottawa,Protected Account,@CBCOttawa,Verified account,https://twitter.com/CBCOttawa,https://www.cbc.ca/news/canada/ottawa,NaN,n,1316,241485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,1066,NaN,NaN,United States of America,NaN,NaN,NaN,English,NaN,NaN,...,http://nfl.com/network,Protected Account,@nflnetwork,Verified account,https://twitter.com/nflnetwork,http://nfl.com,NaN,n,1485,4124366
279,1072,NaN,NaN,Argentina,"Mon, \r\r\nTue, \r\r\nWed, \r\r\nThu, \r\r\nFr...",Daily,https://www.instagram.com/infobaeamerica/,Spanish,NaN,NaN,...,http://infobae.com/america,Protected Account,@infobaeamerica,Verified account,https://twitter.com/InfobaeAmerica,http://infobae.com,https://www.youtube.com/c/infobae,n,651,1270539
280,1080,NaN,NaN,France,NaN,NaN,https://instagram.com/euronews.tv,"Arabic, \r\r\n \r\r\n ...",NaN,NaN,...,http://www.euronews.com/,Protected Account,@euronews,Verified account,https://twitter.com/euronews,http://euronews.com,http://www.youtube.com/euronews,n,2350,459247
281,1082,NaN,NaN,Japan,NaN,NaN,NaN,"English, \r\r\n \r\r\n ...",NaN,NaN,...,http://www6.nhk.or.jp/nhkpr/,Protected Account,@NHK_PR,Verified account,https://twitter.com/NHK_PR,http://nhk.or.jp/nhkworld/,https://www.youtube.com/user/NHKWorld,n,745,2141031


In [22]:
random_rows = [random.randint(0, len(select_df) - 1) for i in range(200)]
random_journos = []

In [23]:
for i in random_rows:
    journos = json.loads(select_df.iloc[i].journalist)
    journo = random.choice(journos)
    random_journos.append(journo)

In [24]:
len(random_journos)

200

#### Fetch articles

In [25]:
nltk.download('stopwords')
nltk.download('wordnet')
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
def clean(doc):
    doc = ftfy.fix_text(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","", normalized)
    y = processed.split()
    return ' '.join(y)

In [32]:
analyze_df = pd.DataFrame(columns = ['name', 'url', 'title', 'description', 'content', 'beat', 'predicted'])

In [33]:
for i in range(len(random_journos)):
    
    try:
    
        name = random_journos[i]['name']
        muckrack = random_journos[i]['profile_url']

        req = Request(muckrack, headers = {'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
        soup = bs.BeautifulSoup(sauce,'lxml')

        beat = soup.find(class_ = 'person-details-beats').find_all('a')
        beat = ", ".join([i.text for i in beat])

        articles = soup.find_all(class_ = "news-story")

        for article in articles:
            title = article.find(class_ = 'news-story-title').text
            description = article.find(class_ = 'news-story-body').text.split('—')[1].strip()
            content = clean(title * 4 + description * 2)

            # print(name, muckrack, title, description, content, beat)
            analyze_df = analyze_df.append({'name': name, 'url': muckrack, 'title': title, 'description': description, 
                                            'content': content, 'beat': beat, 'predicted': '-'}, ignore_index = True)
    
    except:
        pass

In [34]:
analyze_df

,name,url,title,description,content,beat,predicted
0,"Griffiths, Emma",https://muckrack.com/ejgriffiths,Comment: Ideas from Microsoft Ignite 2021,By Peter Buck“Microsoft seeks to power the glo...,comment idea microsoft ignite comment idea mic...,"Australia, Politics",-
1,"Griffiths, Emma",https://muckrack.com/ejgriffiths,"North America: LexFusion Adds CoParse, Kira an...",We speak to co-founder Joe Borstein about the ...,north america lexfusion add coparse kira hotsh...,"Australia, Politics",-
2,"Griffiths, Emma",https://muckrack.com/ejgriffiths,"Comment: The Managed Services Boom - Past, Pre...","By Jeff Catanzaro, Executive Vice President at...",comment managed service boom past present futu...,"Australia, Politics",-
3,"Simplicio, Raisa",https://muckrack.com/raisa-simplicio,Pepê acerta com o Cuibá e Flamengo fica com 20...,Meia se despediu do time carioca nesta quinta ...,pepê acerta com cuibá e flamengo fica com de u...,"Brazil, Sports",-
4,"Simplicio, Raisa",https://muckrack.com/raisa-simplicio,"Artilheiro, Gabigol está há 5 gols de superar ...",Atacante pode quebrar recorde que pertence ao ...,artilheiro gabigol está há gols de superar zic...,"Brazil, Sports",-
...,...,...,...,...,...,...,...
498,"Hebbar, Nistula",https://muckrack.com/nistula,West Bengal Assembly polls | Jorasanko a prest...,BJP is more of a rural party in West Bengal th...,west bengal assembly poll jorasanko prestige b...,"India, Politics",-
499,"Hebbar, Nistula",https://muckrack.com/nistula,West Bengal Assembly polls | Abbas Siddique de...,Mamata’s politics affected brotherhood and ami...,west bengal assembly poll abbas siddique denie...,"India, Politics",-
500,"Dylewski, Adam",https://muckrack.com/adam-dylewski,Londyn Pascal Lajt,Jak dojechać do Hogwartu? Które działy warto o...,londyn pascal lajtlondyn pascal lajtlondyn pas...,"Media, U.S.",-
501,"Dylewski, Adam",https://muckrack.com/adam-dylewski,"Ruda, córka Cwiego. Historia Żydów na warszaws...",Każdy z warszawskich brzegów Wisły opowiada wł...,ruda córka cwiego historia żydów na warszawski...,"Media, U.S.",-


#### Make predictions

In [35]:
iab_mod = pickle.load(open('../Data/IAB/IAB_classifier.p','rb'))
iab_vec = pickle.load(open('../Data/IAB/IAB_vectorizer.p','rb'))
iab_bin = pickle.load(open('../Data/IAB/IAB_binarizer.p','rb'))

C:\Users\ACER\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\ACER\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\ACER\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\ACER\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invali

In [36]:
def iab_taxonomy_v2(text):
    text = clean(text)
    text = iab_vec.transform([text])
    res = iab_bin.inverse_transform(iab_mod.predict(text))
    return res

In [37]:
analyze_df['predicted'] = analyze_df.apply(lambda x: iab_taxonomy_v2(x['content']), axis = 1)

In [42]:
analyze_df.to_csv('../Data/double_randomized_results.csv')